In [14]:
import gmpy2
from Crypto.Util.number import *

nbits = 512
r = 2
s = 3


class Pell_Curve:
    def __init__(self, a, N):
        self.a = a
        self.N = N

    def is_on_curve(self, point):
        if point is None:
            return True
        x, y, z = point
        return (
            x**3 + self.a * y**3 + self.a**2 * z**3 - 3 * self.a * x * y * z
        ) % self.N == 1

    def add(self, P, Q):
        x1, y1, z1 = P
        x2, y2, z2 = Q
        x3 = (x1 * x2 + self.a * (y2 * z1 + y1 * z2)) % self.N
        y3 = (x2 * y1 + x1 * y2 + self.a * z1 * z2) % self.N
        z3 = (y1 * y2 + x2 * z1 + x1 * z2) % self.N
        return (x3, y3, z3)

    def mul(self, P, x):
        Q = (1, 0, 0)
        while x > 0:
            if x & 1:
                Q = self.add(Q, P)
            P = self.add(P, P)
            x >>= 1
        return Q


def psi(p, q, r, s):
    psi1 = p ** (2 * (r - 1)) * q ** (2 * (s - 1)) * (p**2 + p + 1) * (q**2 + q + 1)
    psi2 = p ** (2 * (r - 1)) * q ** (2 * (s - 1)) * (p - 1) ** 2 * (q - 1) ** 2
    psi3 = p ** (2 * (r - 1)) * q ** (2 * (s - 1)) * (p**2 + p + 1) * (q - 1) ** 2
    psi4 = p ** (2 * (r - 1)) * q ** (2 * (s - 1)) * (p - 1) ** 2 * (q**2 + q + 1)
    return (psi1, psi2, psi3, psi4)


def gen(nbits, r, s):
    while 1:
        p = getPrime(nbits)
        q = getPrime(nbits)
        if p % 4 == 1 and q % 4 == 1:
            N = p**r * q**s
            ps = psi(p, q, r, s)
            while 1:
                e = getPrime(17)
                if (
                    gmpy2.gcd(
                        e, p * q * (p**2 + p + 1) * (q**2 + q + 1) * (p - 1) * (q - 1)
                    )
                    == 1
                ):
                    d = []
                    for i in ps:
                        d.append(gmpy2.invert(e, i))
                    pk = (N, e)
                    sk = (p, q, d)
                    return pk, sk


def encrypt(M, N, e):
    xm, ym = M
    M = (xm, ym, 0)
    a = (1 - xm**3) * gmpy2.invert(ym**3, N) % N
    curve = Pell_Curve(a, N)
    if curve.is_on_curve(M):
        return curve.mul(M, e)
    return None


(N, e), sk = gen(nbits, r, s)
m = b'flag{eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee}'
M = (bytes_to_long(m[:len(m)//2]), bytes_to_long(m[len(m)//2:]))
#print(bin(e)[2:])
C = encrypt(M, N, e)
# print(C)
x = var('x')
f1 = C[0]**3+x*C[1]**3+x**2*C[2]**3-3*x*C[0]*C[1]*C[2]-1
solve_mod([f1==0], sk[0])

OverflowError: Python int too large to convert to C ssize_t